In [1]:
from typing import Tuple, Callable

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib.rcsetup import validate_markevery
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
import os
import random
from sympy.physics.quantum.identitysearch import scipy


In [2]:
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

## Helper functions

In [3]:
#DATA_DIR = "/cluster/courses/cil/collaborative_filtering/data"
DATA_DIR = ""

def read_data_df() -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Reads in data and splits it into training and validation sets with a 75/25 split."""

    df = pd.read_csv(os.path.join(DATA_DIR, "data/train_ratings.csv"))

    # Split sid_pid into sid and pid columns
    df[["sid", "pid"]] = df["sid_pid"].str.split("_", expand=True)
    df = df.drop("sid_pid", axis=1)
    df["sid"] = df["sid"].astype(int)
    df["pid"] = df["pid"].astype(int)

    # Split into train and validation dataset
    train_df, valid_df = train_test_split(df, test_size=0.25)
    return train_df, valid_df


def read_data_matrix(df: pd.DataFrame) -> np.ndarray:
    """Returns matrix view of the training data, where columns are scientists (sid) and
    rows are papers (pid)."""

    return df.pivot(index="sid", columns="pid", values="rating").values


def evaluate(valid_df: pd.DataFrame, pred_fn: Callable[[np.ndarray, np.ndarray], np.ndarray]) -> float:
    """
    Inputs:
        valid_df: Validation data, returned from read_data_df for example.
        pred_fn: Function that takes in arrays of sid and pid and outputs their rating predictions.

    Outputs: Validation RMSE
    """

    preds = pred_fn(valid_df["sid"].values, valid_df["pid"].values)
    return root_mean_squared_error(valid_df["rating"].values, preds)


def make_submission(pred_fn: Callable[[np.ndarray, np.ndarray], np.ndarray], filename: os.PathLike):
    """Makes a submission CSV file that can be submitted to kaggle.

    Inputs:
        pred_fn: Function that takes in arrays of sid and pid and outputs a score.
        filename: File to save the submission to.
    """

    df = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))

    # Get sids and pids
    sid_pid = df["sid_pid"].str.split("_", expand=True)
    sids = sid_pid[0]
    pids = sid_pid[1]
    sids = sids.astype(int).values
    pids = pids.astype(int).values

    df["rating"] = pred_fn(sids, pids)
    df.to_csv(filename, index=False)

## Singular value decomposition

For the first method in this introduction, we will make use of the singular value decomposition (SVD) to construct the optimal rank-$k$ approximation (when measuring the Frobenius norm as error), according to the Eckart-Young theorem. Since the matrix needs to be fully observed in order to make use of SVD, we need to impute the missing values. In this case, we impute values with $3$.

In [80]:
def opt_rank_k_approximation(m: np.ndarray, k: int):
    """Returns the optimal rank-k reconstruction matrix, using SVD."""

    assert 0 < k <= np.min(m.shape), f"The rank must be in [0, min(m, n)]"

    U, S, Vh = np.linalg.svd(m, full_matrices=False)

    U_k = U[:, :k]
    S_k = S[:k]
    Vh_k = Vh[:k]

    return np.dot(U_k * S_k, Vh_k)


def matrix_pred_fn(train_recon: np.ndarray, sids: np.ndarray, pids: np.ndarray) -> np.ndarray:
    """
    Input:
        train_recon: (M, N) matrix with predicted values for every (sid, pid) pair.
        sids: (D,) vector with integer scientist IDs.
        pids: (D,) vector with integer paper IDs.
        
    Outputs: (D,) vector with predictions.
    """

    return train_recon[sids, pids]

In [82]:
def impute_int(matrix):
    matrix_filled = matrix.copy()
    return np.nan_to_num(matrix_filled, nan=3)

def impute_mean(matrix):
    matrix_filled = matrix.copy()
    for i in range(matrix.shape[0]):
        row_means = (matrix_filled[i][~np.isnan(matrix[i])]).mean()
        matrix_filled[i][np.isnan(matrix[i])] = row_means
    return matrix_filled

def impute_global_mean(matrix):
    global_mean = matrix[~np.isnan(matrix)].mean()
    matrix_filled = matrix.copy()
    matrix_filled[np.isnan(matrix)] = global_mean
    return matrix_filled

# Assuming read_data_df(), read_data_matrix(), evaluate() are defined elsewhere
train_df, valid_df = read_data_df()
train_mat_original = read_data_matrix(train_df)

def run_experiment(impute_fn, impute_name):
    train_mat = impute_fn(train_mat_original)
    train_recon = opt_rank_k_approximation(train_mat, k=2)
    pred_fn = lambda sids, pids: matrix_pred_fn(train_recon, sids, pids)
    
    test_score = evaluate(train_df, pred_fn)
    val_score = evaluate(valid_df, pred_fn)
    print(f"{impute_name} | Train RMSE: {test_score:.4f}, Validation RMSE: {val_score:.4f}")

run_experiment(impute_int, "Impute with 3")
run_experiment(impute_mean, "Impute with Row Mean")
run_experiment(impute_global_mean, "Impute with Global Mean")


Impute with 3 | Train RMSE: 1.2005, Validation RMSE: 1.2068
Impute with Row Mean | Train RMSE: 0.9152, Validation RMSE: 0.9344
Impute with Global Mean | Train RMSE: 0.9781, Validation RMSE: 0.9876


## ALS

In [94]:
factor = 10
lambda_reg = 1
num_iterations = 10

train_df, valid_df = read_data_df()
train_mat = read_data_matrix(train_df) # num_pid x num_sid

num_pid = train_mat.shape[0]
num_sid = train_mat.shape[1]

U = np.random.normal(size=(num_pid, factor))
V = np.random.normal(size=(num_sid, factor))

mask = ~np.isnan(train_mat)

Updating the two Matrices $U$ and $V$ alternately, i.e.

$$u_i = \left(\sum_{j}\omega_{ij}\cdot v_j v_j^T + \lambda \cdot I\right)^{-1}\left(\sum_j\omega_{ij} \cdot a_{ij}v_j\right)$$
and 
$$v_j = \left(\sum_{i}\omega_{ij}\cdot u_iu_i^T + \lambda \cdot I\right)^{-1}\left(\sum_i\omega_{ij} \cdot a_{ij}u_i\right)$$

In [95]:
for it in range(num_iterations):
    # Fix V, solve for U
    for i in range(num_pid):
        V_i = V[mask[i]]
        train_mat_i = train_mat[i][mask[i]]
        A = V_i.T @ V_i + lambda_reg * np.eye(factor)
        b = V_i.T @ train_mat_i
        U[i] = np.linalg.solve(A, b)

    # Fix U, solve for V
    for j in range(num_sid):
        U_j = U[mask[:, j]]
        train_mat_j = train_mat[:, j][mask[:, j]]
        A = U_j.T @ U_j + lambda_reg * np.eye(factor)
        b = U_j.T @ train_mat_j
        V[j] = np.linalg.solve(A, b)

    # Compute loss
    recon = U @ V.T
    squared_error = ((train_mat - recon)[mask])**2
    mse = np.sum(squared_error) / np.sum(mask)
    rmse = np.sqrt(mse)
    
    pred_fn = lambda sids, pids: matrix_pred_fn(recon, sids, pids)
    test_score = evaluate(train_df, pred_fn)
    val_score = evaluate(valid_df, pred_fn)

    print(f"Iteration {it + 1}/{num_iterations}, RMSE: {test_score:.4f}, Validation RMSE: {val_score:.4f}")

Iteration 1/10, RMSE: 2.5229, Validation RMSE: 2.7304
Iteration 2/10, RMSE: 0.8023, Validation RMSE: 0.9174
Iteration 3/10, RMSE: 0.7774, Validation RMSE: 0.9119
Iteration 4/10, RMSE: 0.7659, Validation RMSE: 0.9079
Iteration 5/10, RMSE: 0.7596, Validation RMSE: 0.9053
Iteration 6/10, RMSE: 0.7559, Validation RMSE: 0.9039
Iteration 7/10, RMSE: 0.7536, Validation RMSE: 0.9032
Iteration 8/10, RMSE: 0.7521, Validation RMSE: 0.9029
Iteration 9/10, RMSE: 0.7510, Validation RMSE: 0.9028
Iteration 10/10, RMSE: 0.7502, Validation RMSE: 0.9028


## Variational Autoencoder

A **Variational Autoencoder (VAE)** is a generative model that learns to encode input data into a **latent space** represented by a probability distribution. Instead of mapping inputs to fixed points, the encoder outputs parameters (mean and variance) of this distribution. During training, latent vectors are sampled from these distributions, allowing the decoder to reconstruct the input. 

In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


class RatingDataset(Dataset):
    def __init__(self, rating_matrix, impute_func):
        self.original_data = rating_matrix
        self.data = impute_func(rating_matrix)
        self.mask = ~torch.isnan(torch.tensor(rating_matrix)).to(torch.bool) 

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        return self.data[idx], self.mask[idx]


class VAE_CF(nn.Module):
    def __init__(self, num_items, hidden_dim=20, latent_dim=10, dropout=0.5):
        super(VAE_CF, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(num_items, hidden_dim),
            nn.Tanh(),
            nn.Dropout(dropout),
        )
        self.mu_layer = nn.Linear(hidden_dim, latent_dim)
        self.logvar_layer = nn.Linear(hidden_dim, latent_dim)

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.Tanh(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_items)
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu = self.mu_layer(h)
        logvar = self.logvar_layer(h)
        z = self.reparameterize(mu, logvar)
        out = self.decoder(z)
        return out, mu, logvar

    def loss_function(self, recon_x, x, mu, logvar, mask):
        # Reconstruction loss
        recon_loss = F.mse_loss(recon_x[mask], x[mask])
        # KL divergence
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return recon_loss + kl_loss, recon_loss.item(), kl_loss


In [ ]:
# Define imputation strategies
def impute_int(matrix):
    matrix_filled = matrix.copy()
    return torch.tensor(np.nan_to_num(matrix_filled, nan=3), dtype=torch.float32)

def impute_mean(matrix):    
    matrix_filled = matrix.copy()
    for i in range(matrix.shape[0]):
        row_means = (matrix_filled[i][~np.isnan(matrix[i])]).mean()
        matrix_filled[i][np.isnan(matrix[i])] = row_means
    return torch.tensor(matrix_filled, dtype=torch.float32)

def impute_global_mean(matrix):
    global_mean = matrix[~np.isnan(matrix)].mean()
    matrix_filled = matrix.copy()
    matrix_filled[np.isnan(matrix)] = global_mean
    return torch.tensor(matrix_filled, dtype=torch.float32)

# Load data
train_df, valid_df = read_data_df()
train_mat = read_data_matrix(train_df)
valid_mat = read_data_matrix(valid_df)

# Define imputation strategies to test
imputation_strategies = {
    "int (3)": impute_int,
    "row_mean": impute_mean,
    "global_mean": impute_global_mean,
}

# Evaluate each strategy
for name, impute_func in imputation_strategies.items():
    print(f"\n>>> Imputation strategy: {name}")

    # Prepare datasets
    train_dataset = RatingDataset(train_filled, impute_func)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    model = VAE_CF(num_items=train_mat.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(10):
        model.train()
        total_loss = 0
        total_data = 0
        for batch, batch_mask in train_loader:
            recon, mu, logvar = model(batch)
            loss, recon_loss, kl = model.loss_function(recon, batch, mu, logvar, batch_mask)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += recon_loss * len(batch)
            total_data += len(batch)
    
        model.eval()
        with torch.no_grad():
            # Use imputed train matrix to get representations
            train_tensor = impute_func(train_mat)
            recon, _, _ = model(train_tensor)
    
            # Evaluate only on valid entries in validation matrix
            valid_mask = ~np.isnan(valid_mat)
            valid_true = torch.tensor(valid_mat[valid_mask], dtype=torch.float32)
            valid_pred = recon[valid_mask]
    
            val_rmse = torch.sqrt(torch.mean((valid_true - valid_pred) ** 2)).item()
            print(f"Epoch {epoch+1:02d} | Train RMSE: {(total_loss / total_data) ** 0.5:.4f}, Validation RMSE: {val_rmse:.4f}")

## Learned embeddings

Next, we will take a machine learning view of the problem. To each scientist and paper, we assign a $d$-dimensional embedding and we predict the rating that the scientist gives the paper to be their dot product. More formally, let $\vec{s}_i$ be a scientist embedding and $\vec{p}_j$ be a paper embedding. Then, we make the following rating prediction for this pair: $$\tilde{r}_{ij} = \langle \vec{s}_i, \vec{p}_j \rangle.$$ We view these embeddings as our learnable parameters and train them as we would any other model using the squared error loss function: $$\ell(\theta) = \frac{1}{2} |\langle \vec{s}_i, \vec{p}_j \rangle - r_{ij}|^2,$$ where $\theta = \{ \vec{s}_i \}_{i=1}^n \cup \{ \vec{p}_j \}_{j=1}^m$. The following is an implementation of this method.## Embedding

In [96]:
def get_dataset(df: pd.DataFrame) -> torch.utils.data.Dataset:
    """Conversion from pandas data frame to torch dataset."""

    sids = torch.from_numpy(df["sid"].to_numpy())
    pids = torch.from_numpy(df["pid"].to_numpy())
    ratings = torch.from_numpy(df["rating"].to_numpy()).float()
    return torch.utils.data.TensorDataset(sids, pids, ratings)

In [97]:
class EmbeddingDotProductModel(nn.Module):
    def __init__(self, num_scientists: int, num_papers: int, dim: int):
        super().__init__()

        # Assign to each scientist and paper an embedding
        self.scientist_emb = nn.Embedding(num_scientists, dim)
        self.paper_emb = nn.Embedding(num_papers, dim)

    def forward(self, sid: torch.Tensor, pid: torch.Tensor) -> torch.Tensor:
        """
        Inputs:
            sid: [B,], int
            pid: [B,], int
        
        Outputs: [B,], float
        """

        # Per-pair dot product
        return torch.sum(self.scientist_emb(sid) * self.paper_emb(pid), dim=-1)

In [98]:
# Define model (10k scientists, 1k papers, 32-dimensional embeddings) and optimizer
device = torch.device("mps")

model = EmbeddingDotProductModel(10_000, 1_000, 32).to(device)
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

In [99]:
train_dataset = get_dataset(train_df)
valid_dataset = get_dataset(valid_df)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [102]:
NUM_EPOCHS = 5
for epoch in range(NUM_EPOCHS):
    # Train model for an epoch
    total_loss = 0.0
    total_data = 0
    model.train()
    for sid, pid, ratings in train_loader:
        sid = sid.to(device)
        pid = pid.to(device)
        ratings = ratings.to(device)

        # Make prediction and compute loss
        pred = model(sid, pid)
        loss = F.mse_loss(pred, ratings)

        # Compute gradients w.r.t. loss and take a step in that direction
        optim.zero_grad()
        loss.backward()
        optim.step()

        # Keep track of running loss
        total_data += len(sid)
        total_loss += len(sid) * loss.item()

    # Evaluate model on validation data
    total_val_mse = 0.0
    total_val_data = 0
    model.eval()
    for sid, pid, ratings in valid_loader:
        # Move data to GPU
        sid = sid.to(device)
        pid = pid.to(device)
        ratings = ratings.to(device)

        # Clamp predictions in [1,5], since all ground-truth ratings are
        pred = model(sid, pid).clamp(1, 5)
        mse = F.mse_loss(pred, ratings)

        # Keep track of running metrics
        total_val_data += len(sid)
        total_val_mse += len(sid) * mse.item()

    print(f"[Epoch {epoch+1}/{NUM_EPOCHS}] Train loss={total_loss / total_data:.3f}, Valid RMSE={(total_val_mse / total_val_data) ** 0.5:.3f}")

[Epoch 1/5] Train loss=0.849, Valid RMSE=0.949
[Epoch 2/5] Train loss=0.828, Valid RMSE=0.946
[Epoch 3/5] Train loss=0.803, Valid RMSE=0.943
[Epoch 4/5] Train loss=0.778, Valid RMSE=0.943
[Epoch 5/5] Train loss=0.754, Valid RMSE=0.944


In [101]:
pred_fn = lambda sids, pids: model(torch.from_numpy(sids).to(device), torch.from_numpy(pids).to(device)).clamp(1, 5).cpu().numpy()

# Evaluate on validation data
with torch.no_grad():
    val_score = evaluate(valid_df, pred_fn)

print(f"Validation RMSE: {val_score:.3f}")

Validation RMSE: 0.955
